# Fundamentals of Data Analytics HW2
by 交管108_張仲宇_H54046070

---
## 1. Question
誰會被升等商務艙？

---

## 2. Attributes/Features
旅客屬性：
1. 是不是很有錢
2. 是不是航空公司會員
3. 是不是一個人旅行
4. 是不是太晚報到

[本屬性設計參考](https://www.funtime.com.tw/blog/funtime/%E6%90%AD%E6%A9%9F%E5%BF%85%E7%9C%8B-%E5%85%8D%E8%B2%BB%E5%8D%87%E7%AD%89%E5%95%86%E5%8B%99%E8%89%99%E5%B0%8F%E6%8A%80%E5%B7%A7)

---
## 3. Designed Rules
以下為樹狀圖：

[連結](https://imgur.com/MgBkJ5r)
<img src="Diagram.jpg">
---
## 4. Generate Data

In [1]:
import pandas as pd

In [2]:
passenger = pd.read_csv('passenger.csv')
passenger.head()

,Rich,Member,Alone,Late,BusinessClass
0,1,1,0,0,1
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,1,0,0,0,1


### 檔案說明：
這邊自己亂填產生了**100筆乘客資料**，以下是各個變數說明：
- Rich: 是不是很有錢，0:沒錢 1:有錢
- Member: 是不是航空公司會員，0:不是會員 1:是會員
- Alone: 是不是一個人旅行，0:不是一個人 1:一個人
- Late: 是不是太晚報到，0:不會太晚 1:太晚報到
- BusinessClass: 最後是否搭乘商務艙，0:經濟艙 1:商務艙

In [3]:
passenger['BusinessClass'].value_counts()

0    79
1    21
Name: BusinessClass, dtype: int64

---
## 5. Construct Decision Tree

接下來利用Scikit-Learn產生決策樹來比較看看和自己設計的差異

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz
import pydotplus

In [5]:
y = passenger['BusinessClass'].values
passenger = passenger.drop('BusinessClass', 1)

y

array([1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [6]:
dtree = DecisionTreeClassifier(max_depth=4)
dtree.fit(passenger ,y)

dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(passenger),
                class_names=['Economic','Business'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("tree.pdf")

True

<img src="tree.png">

## 6. Compare the Rules
從產生的決策樹來看可以發現他精準的將所有最後乘坐商務艙的旅客分出來了，gini值都非常低，

不過有一個不同的地方是他第二步先從"Alone"下手，而不是我原本設計的"Member"。

可能是這組資料的結構和設計真的太簡單，而且**設計時沒有例外，完全遵照自己的規則去填結果**，

所以對於機器學習來說簡直是小兒科。

---
#### 接下來試著在乘客資料中加入一些noise，看看會跑出什麼結果。


In [7]:
passenger_noise = pd.read_csv('passenger_noise.csv')

In [8]:
passenger_noise['BusinessClass'].value_counts()

0    77
1    23
Name: BusinessClass, dtype: int64

In [9]:
y = passenger_noise['BusinessClass'].values
passenger_noise = passenger_noise.drop('BusinessClass', 1)

y

array([1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [16]:
dtree = DecisionTreeClassifier(max_depth=4)
dtree.fit(passenger_noise ,y)

dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(passenger_noise),
                class_names=['Economic','Business'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("tree_noise.pdf")

True

<img src="tree_noise.png">

加入一些noise後，可以發現決策樹變得更複雜了，

判斷出來的結果也變成不是完全準確。

---
由此可知原始資料的純度與精準度也是非常重要的，

如果能夠在一開始就把資料好好的做一番分析與清洗，

那麼接下來的分析預測工作也會變得更簡單、更準確！